In [123]:
import pdfplumber, re, redis, numpy as np, json, requests
from sentence_transformers import SentenceTransformer
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from collections import deque

In [124]:
# ============================================================
# 1️⃣ Redis & 모델 초기화
# ============================================================
r = redis.Redis(host="localhost", port=6379, decode_responses=False)
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

CACHE_INDEX = "cache_index"   # 캐시 인덱스
RAG_INDEX = "qa2_index"        # RAG 인덱스
user_cache = deque(maxlen=5)  # Dynamic Cache (최근 5개)

In [125]:
# ------------------------------------------------------------
# 2️⃣ 임베딩 함수 (정규화 비활성화: Redis COSINE과 호환)
# ------------------------------------------------------------
def embed(text: str):
    # 🚀 중요: normalize_embeddings=False → Redis COSINE 계산 정확도 향상
    emb = model.encode(text, normalize_embeddings=False)
    return np.array(emb, dtype=np.float32).tobytes()

In [126]:
# ------------------------------------------------------------
# 3️⃣ 캐시 인덱스 초기화 (기존 인덱스 삭제 후 재생성)
# ------------------------------------------------------------
def init_cache_index(force_recreate=True):
    if force_recreate:
        try:
            r.ft(CACHE_INDEX).dropindex(delete_documents=True)
            print("🗑️ 기존 cache_index 삭제 완료")
        except Exception:
            pass

    try:
        r.ft(CACHE_INDEX).info()
        print("ℹ️ cache_index 이미 존재 (재사용)")
    except:
        dim = len(model.encode("차원 확인", normalize_embeddings=False))
        r.ft(CACHE_INDEX).create_index(
            fields=[
                VectorField("embedding", "FLAT", {
                    "TYPE": "FLOAT32",
                    "DIM": dim,
                    "DISTANCE_METRIC": "COSINE"   # ✅ COSINE + 비정규화 벡터 조합
                }),
                TextField("text"),
                TextField("source")
            ],
            definition=IndexDefinition(prefix=["cache:"], index_type=IndexType.HASH)
        )
        print("✅ cache_index 인덱스 생성 완료")

init_cache_index()

🗑️ 기존 cache_index 삭제 완료
✅ cache_index 인덱스 생성 완료


In [127]:
# ------------------------------------------------------------
# 4️⃣ PDF Q–A 파싱 (본문 + 표 포함)
# ------------------------------------------------------------
PDF_PATH = "/Users/yoodongseok/Desktop/rag_project/data/2024 관세행정 민원상담 사례집.pdf"

def extract_qa_pairs(pdf_path):
    qa_pairs = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_idx, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            lines = text.split("\n") if text else []

            # 표 추출
            tables = page.extract_tables()
            table_texts = []
            for table in tables:
                rows = [" | ".join([cell if cell else "" for cell in row]) for row in table]
                table_texts.append("\n".join(rows))
            table_text_block = "\n\n[표 데이터]\n" + "\n\n".join(table_texts) if table_texts else ""

            merged_text = text + table_text_block

            current_q, current_a = None, []
            for line in merged_text.split("\n"):
                line = line.strip()
                if re.match(r".*(\?|궁금합니다\.?|알려주세요\.?|무엇인가요\.?|어떻게.*|대해 설명.*|요약.*)$", line):
                    if current_q and current_a:
                        qa_pairs.append({
                            "question": current_q,
                            "answer": "\n".join(current_a).strip()
                        })
                    current_q = line
                    current_a = []
                elif current_q:
                    current_a.append(line)

            if current_q and current_a:
                qa_pairs.append({
                    "question": current_q,
                    "answer": "\n".join(current_a).strip()
                })

    return qa_pairs

In [128]:
# ------------------------------------------------------------
# 5️⃣ Pre-Cache (PDF → Redis 저장)
# ------------------------------------------------------------
def pre_cache_pdf(pdf_path):
    qa_list = extract_qa_pairs(pdf_path)
    print(f"📘 PDF에서 {len(qa_list)}개의 QA 추출 완료")

    for i, qa in enumerate(qa_list):
        key = f"cache:pdf:{i}"
        r.hset(key, mapping={
            "embedding": embed(qa["question"]),
            "text": qa["answer"],
            "source": "pdf_pre_cache"
        })
    print(f"💾 Redis에 {len(qa_list)}개 Pre-Cache 저장 완료")

pre_cache_pdf(PDF_PATH)


📘 PDF에서 913개의 QA 추출 완료
💾 Redis에 913개 Pre-Cache 저장 완료


In [129]:
# ------------------------------------------------------------
# 6️⃣ 캐시 검색 (CAG)
# ------------------------------------------------------------
def check_cache(user_query, k=3, threshold=0.6):
    q_emb = embed(user_query)
    q = Query(f"*=>[KNN {k} @embedding $vec AS score]") \
        .return_fields("text", "source", "score") \
        .sort_by("score") \
        .dialect(2)

    try:
        res = r.ft(CACHE_INDEX).search(q, query_params={"vec": q_emb})
    except Exception as e:
        print("❌ 캐시 검색 오류:", e)
        return None

    if not res.docs:
        print("❌ 캐시에서 유사 문서 없음")
        return None

    # 유사도 계산
    sim = 1 - float(res.docs[0].score)
    print(f"📊 유사도 점수: {sim:.2f}")
    if sim >= threshold:
        print(f"⚡ 캐시 HIT (유사도 {sim:.2f}) [source={res.docs[0].source}]")
        return res.docs[0].text
    else:
        print(f"❌ 캐시 MISS (유사도 {sim:.2f} < {threshold})")
    return None

In [130]:
# ------------------------------------------------------------
# 7️⃣ Dynamic Cache 저장 (최근 5개 유지)
# ------------------------------------------------------------
def save_dynamic_cache(query, answer):
    key = f"cache:dyn:{abs(hash(query)) % (10**8)}"
    r.hset(key, mapping={
        "embedding": embed(query),
        "text": answer,
        "source": "dynamic_cache"
    })
    user_cache.append(key)
    if len(user_cache) > user_cache.maxlen:
        oldest = user_cache.popleft()
        r.delete(oldest)
        print(f"🗑️ 오래된 캐시 삭제: {oldest}")
    print(f"💾 Dynamic Cache 저장: {query[:30]}...")

In [131]:
# ------------------------------------------------------------
# 8️⃣ RAG 검색 (qa_index에서 검색)
# ------------------------------------------------------------
def search_rag_context(query, k=3):
    q_emb = embed(query)
    q = Query(f"*=>[KNN {k} @embedding $vec AS score]") \
        .return_fields("answer", "score") \
        .sort_by("score") \
        .dialect(2)
    try:
        res = r.ft(RAG_INDEX).search(q, query_params={"vec": q_emb})
        return [doc.answer for doc in res.docs]
    except Exception as e:
        print("❌ RAG 검색 오류:", e)
        return []

In [132]:
# ------------------------------------------------------------
# 9️⃣ Llama3.2 (Ollama) 호출
# ------------------------------------------------------------
def call_llm_with_context(query, contexts):
    context_text = "\n\n".join(contexts)
    prompt = f"""
너는 관세행정 민원상담 전문 챗봇이야.
아래 문서를 참고해 질문에 정확히 답변해줘.

[문서 내용]
{context_text}

[질문]
{query}
"""
    url = "http://localhost:11434/api/generate"
    payload = {"model": "llama3.2:3b", "prompt": prompt}

    response_text = ""
    with requests.post(url, json=payload, stream=True) as r_:
        for line in r_.iter_lines():
            if line:
                data = json.loads(line.decode("utf-8"))
                if "response" in data:
                    response_text += data["response"]
                if data.get("done", False):
                    break
    return response_text.strip()

In [133]:
# ------------------------------------------------------------
# 🔟 전체 파이프라인 (CAG → RAG → LLM → Dynamic Cache)
def ask(query):
    print(f"\n👤 사용자 질문: {query}")

    cached = check_cache(query)
    if cached:
        return f"(캐시 응답)\n{cached}"

    print("💡 캐시 MISS → RAG 검색 중...")
    contexts = search_rag_context(query)
    if not contexts:
        return "❌ 관련 문서를 찾지 못했습니다."

    print("🤖 Llama3.2 응답 생성 중...")
    answer = call_llm_with_context(query, contexts)
    save_dynamic_cache(query, answer)

    return f"(새로 생성된 응답)\n{answer}"


In [134]:
print(ask("여름방학 때 친구들과 제주도 여행을 가려고 합니다. 제주도 면세점에서 담배를 구입할 수 있다고 들었습니다. 요즘 담배값이 많이 올라서 기회가 되면 최대한 구매하고 싶습니다. 얼마나 살 수 있나요?"))


👤 사용자 질문: 여름방학 때 친구들과 제주도 여행을 가려고 합니다. 제주도 면세점에서 담배를 구입할 수 있다고 들었습니다. 요즘 담배값이 많이 올라서 기회가 되면 최대한 구매하고 싶습니다. 얼마나 살 수 있나요?
📊 유사도 점수: 0.46
❌ 캐시 MISS (유사도 0.46 < 0.6)
💡 캐시 MISS → RAG 검색 중...
🤖 Llama3.2 응답 생성 중...


KeyboardInterrupt: 

In [135]:
print(ask("자가용 보세창고를 운영하고 싶습니다. 관련 절차가 궁금합니다."))


👤 사용자 질문: 자가용 보세창고를 운영하고 싶습니다. 관련 절차가 궁금합니다.
📊 유사도 점수: 1.00
⚡ 캐시 HIT (유사도 1.00) [source=pdf_pre_cache]
(캐시 응답)
자가용보세창고를 운영하고자 하는 자는 ｢관세법｣ 제174조 및 제175조,
｢특허보세구역운영에 관한 고시｣ 제3조, 제13조, 제14조의 요건을
갖추고, 제5조에 따른 서류를 세관장에게 제출하여 특허심사를 받아야 함
[고시 제3조 운영인의 자격]
- 관세법 제175조의 결격사유에 해당하지 않을 것
- 체납된 관세 및 내국세가 없을 것
- 보세사 자격 취득 또는 1명 이상의 보세사 채용
- 위험물품을 장치하는 경우 관계행정기관장의 허가 등
[고시 제13조 특허요건]
자가화물을 장치하려는 경우. 다만, 아래 물품으로서 보세화물 감시단속에
문제가 있는 경우 특허하지 않을 수 있음
･
- 소량 고가물품(귀금속 등)
- 고세율 물품(농산물 등)
- 위와 유사한 물품
[고시 제14조 시설요건]
･ ･
- 장치 보관되는 물품의 종류 특성에 따라 필요한 면적 확보
- 시설요건은 고시 제10조제1항제2호부터 제9호까지, 제10조 제2항, 제10
조제4항 및 제11조를 준용
[고시 제5조 특허신청시 제출서류]
- 특허보세구역 특허신청서
- 임대차계약서(임차의 경우에만 해당함)
- 해당 보세구역의 운영과 관계있는 임원의 인적사항
- 위험물품을 장치하는 경우 관계행정기관장의 허가 등
- 보세구역의 도면 및 부근 위치도
관련법령
● ｢관세법｣ 제174조, 제175조 및 ｢특허보세구역운영에 관한 고시｣ 제3조, 제13조, 제14조 등
406|관세행정 민원상담 사례집


In [136]:
print(ask("요즘 담배값이 많이 올라서 기회가 되면 최대한 구매하고 싶습니다. 얼마나 살 수 있나요?"))


👤 사용자 질문: 요즘 담배값이 많이 올라서 기회가 되면 최대한 구매하고 싶습니다. 얼마나 살 수 있나요?
📊 유사도 점수: 0.74
⚡ 캐시 HIT (유사도 0.74) [source=pdf_pre_cache]
(캐시 응답)
여행자가 휴대반입하는 담배는 궐련 200개비, 엽궐련 50개비, 전자담배
니코틴 용액(니코틴함량 1%이하) 20ml 이하 등 한가지 품목에 대해서만
면세 처리됩니다.
- 단, 만 19세 미만인 사람(만 19세가 되는 해의 1월 1일을 맞이한 사람은
제외)이 반입하는 담배는 면세대상에 포함되지 않습니다.
담배는 여행자 휴대품 기본면세범위(US$800)에서 제외되어 별도로
면세처리 됩니다. 면세범위 초과 시 관세, 부가가치세, 개별소비세 외에
지방세(담배소비세, 지방교육세)가 부과됩니다.
담배의 종류 종류별 면세범위 비 고
• 궐련 • 200개비
• 엽궐련 • 50개비
• 1인당 기본면세범위(US$800)와는
• 궐련형 200개비
별도로 면세
• 전자담배 • 니코틴용액* 20㎖
• 수량기준으로 가격제한 없음
• 기타 유형 110g
• 한 종류만 해당
• 그 밖의 담배 • 250g
* 니코틴 용액 : 니코틴 함량 1% 미만일 경우만 가능(화학물질관리법) \
관련법령
● 관세법 시행규칙 제48조
PART 1 통 관|205

[표 데이터]

통
관

담배의 종류 | 종류별 면세범위 | 비 고
| • 200개비 |
| • 50개비 |
| • 궐련형 200개비
• 니코틴용액* 20㎖
• 기타 유형 110g |
| • 250g |


In [137]:
print(ask("해외 구매 식품 국내 휴대 반입시 유의사항"))


👤 사용자 질문: 해외 구매 식품 국내 휴대 반입시 유의사항
📊 유사도 점수: 0.71
⚡ 캐시 HIT (유사도 0.71) [source=pdf_pre_cache]
(캐시 응답)
건강기능(보조)식품이나 일반의약품인 경우의 자가사용 인정기준은 각각
총 6병(6병을 초과하는의약품인 경우에는 의약품 용법상 3개월 복용량)
이며, 면세통관범위를 초과하는 경우에는 요건확인대상입니다. 다만,
환자가 질병치료를 위해 수입하는 건강기능식품은 의사의 소견서 등에
의거 타당한 범위내에서 요건확인을 면제해주고 있습니다.
다만, 다음의 물품은 요건확인대상물품입니다.
- CITES규제물품(사향 등) 성분이 함유된 물품
- 식품의약품안전처장의 수입 불허 또는 유해의약품 통보를 받은 품목
- 외포장상 성분표시가 불명확한 물품
- 에페드린, 놀에페드린, 슈도에페드린, 에르고타민, 에르고메트린 함유 단일
완제의약품의 경우에는, 면세범위내의 물품이라 하더라도 해당법령에서 정한
수입요건을 충족하여야 합니다.
건강기능식품의 경우 우피유래 성분이나 의약품성분이 포함되어 있을 경우
통관이 제한됩니다. 우선 수입하고자하는 물품의 원료 성분을 확인한 뒤
식품의약품안전처에 질의하여 의약품 성분 또는 유해성분 함유여부를
확인해 보시고 수입하시기 바랍니다.
\
관련법령
● 관세법 제226조(허가·승인 등의 증명 및 확인)
PART 1 통 관|253

[표 데이터]

통
관


In [141]:
print(ask("출국할 때 가지고 갔던 국내에서 구입한 물품을 입국할 때 면세 받고 싶습니다."))


👤 사용자 질문: 출국할 때 가지고 갔던 국내에서 구입한 물품을 입국할 때 면세 받고 싶습니다.
📊 유사도 점수: 0.73
⚡ 캐시 HIT (유사도 0.73) [source=pdf_pre_cache]
(캐시 응답)
출국 시 국내 면세점(또는 기내)에서 구매한 물품을 구매자가 직접 휴대
입국하여 교환 또는 환불하고자 하는 경우 구매자가 자진신고(교환·환불
하려는 물품가격 총액이 여행자 휴대품 면세범위 이하인 경우는 제외)한 후
유치 또는 과세통관을 한 후 구매처로 직접 교환, 환불 요청을 하시면
됩니다.
- 다만, 유치한 후 교환된 물품은 구매자가 출국하는 때 인도됩니다.
교환 및 반품 가능 기간, 절차 등은 구매처로 문의하셔야 합니다.
\
관련법령
● 기타
PART 1 통 관|207

[표 데이터]

통
관


In [144]:
print(ask("향수의 면세범위는 어떻게 되나요?"))


👤 사용자 질문: 향수의 면세범위는 어떻게 되나요?
📊 유사도 점수: 1.00
⚡ 캐시 HIT (유사도 1.00) [source=pdf_pre_cache]
(캐시 응답)
향수의 여행자 휴대품 면세범위는 100㎖ 또는 이에 준하는 부피 또는
중량 이내의 것(예: 100g, 3.4oz 등, 수량 제한 없음)이며, 이 경우 관세
및 부가가치세 등이 면제됩니다.
- 향수는 여행자 휴대품 기본면세범위(US$800)에서 제외되어 별도로 면세
처리됩니다.
\
관련법령
● 관세법 시행규칙 제48조
PART 1 통 관|215

[표 데이터]

통
관


In [145]:
print(ask("재수출 조건부 일시반입 물품은 무엇인가요?"))


👤 사용자 질문: 재수출 조건부 일시반입 물품은 무엇인가요?
📊 유사도 점수: 1.00
⚡ 캐시 HIT (유사도 1.00) [source=pdf_pre_cache]
(캐시 응답)
재수출 조건부 일시반입 제도는 우리나라를 일시 입국하는 여행자가
사용할 물품에 대해 재수출하는 조건으로 세금을 면제해주는 ‘조건부
면세제도’입니다.
대상자 및 허용범위는 외국거주자 중 우리나라를 일시방문하는 자이거나
우리나라 국적 소유자 중 교포, 유학생, 해외근무자 등 1년이상 외국에
거주하는 자의 신변용품, 신변장식용품, 직업용품으로 한정(회사용물품은
불가)하며, 반드시 재수출할 물품에 한해 엄격히 적용하고 있습니다.
재수출면세기간은 1년 이내 최초 출국일까지이며, 1년의 범위 내에서
연장이 가능하며, 반출기한 연장은 횟수에 제한은 없으나 전체 연장기간은
1년을 초과할 수 없어 입국시부터 최초 출국시까지 최장 2년간 면세가
가능합니다.
여행자는 재수출 조건부 일시반입 허용일 이후 최초 출국하는 날에 일시반입한
물품과 재수출조건 일시반입물품 확인서를 가지고 출국장에서 세관 직원에게
확인을 받고 반출하여야 합니다.
타인이 아닌 면세받은 여행자가 출국 시 반출확인을 직접 받아야 하며,
기한 내 재반출을 이행하지 않은 경우 최초 조건부 면세받은 세액 전액과
미반출에 대한 가산세 20%를 부과하여 추징합니다.
관련법령
● 관세법 제97조제1항제1호
● 관세법 시행령 제115조제1항
● 여행자 및 승무원 휴대품통관에 관한 고시 제60조~제63조
216|관세행정 민원상담 사례집


In [146]:
print(ask("면세범위를 초과하는 담배를 통관할 때 부과되는 세금은 무엇인가요?"))


👤 사용자 질문: 면세범위를 초과하는 담배를 통관할 때 부과되는 세금은 무엇인가요?
📊 유사도 점수: 1.00
⚡ 캐시 HIT (유사도 1.00) [source=pdf_pre_cache]
(캐시 응답)
\
여행자 면세범위를 초과하는 담배를 과세통관하는 경우 관세, 부가가치세,
개별소비세 외에 지방세(담배소비세, 지방교육세)가 부과되며, 지방세는
국세와 별도로 고지서를 발급하고 있습니다.
담배 종류별 여행자 휴대품 면세범위(1인당)
‣ 궐련 200개비, 엽궐련 50개비
‣ 전자담배 : 궐련형 200개비, 니코틴용액(1%미만) 20ml, 기타유형 110g
‣ 그밖의 담배 250g
※ 1종류만 면세 가능, 기본면세범위와 별도 면세, 미성년자는 면세 제외
담배란 담배사업법 제2조에 따라 연초의 잎을 원료의 전부 또는 일부로
하여 피우거나 빨거나 증거로 흡입하거나 씹거나 냄새 맡기에 적합한
상태로 제조한 것을 의미하며, 담배 종류에 따른 세종 및 세율은 아래
표와 같습니다.
종가세(%) 종량세(원)
국세 지방세
구분 종류 부가 과세
관세 세 단위 개별 담배 지방
소비세 소비세 교육세
궐련(필터담배) 20개비당 594 1,007 443
피 파이프담배 40 1g당 21 36 16
엽궐련(시가) 1g당 61 103 45
우
각련 1g당 21 36 16
는
액상 8 용액 1ml당 370 628 276
담 전자 10
궐련형 40 20개비당 529 897 395
배 담배 기타형 40 1g 51 88 39
물담배 1g당 422 715 315
씹거나 머금는 담배 40 1g당 215 364 160
냄새맡는 담배(코담배) 1g당 15 26 11
‣ 지방교육세 : 담배소비세액의 10,000분의 4,339(43.99%) 「지방세법 제151조」
‣ 부가가치세의 면세 : 담배사업법 제2조의 규정에 의한 담배로서 판매가격이 200원이
하의 것 또는 담배사업법 제19조 규정에 의한 특수용 담배인 경우 면세, 그 외는
모두 과세대상
관련법령
● 관세법 시행규칙 제48조제3항
218|관세행정 민원상

In [147]:
print(ask("외국에서 흙을 휴대품으로 가지고 오고 싶은데 가능한가요?"))


👤 사용자 질문: 외국에서 흙을 휴대품으로 가지고 오고 싶은데 가능한가요?
📊 유사도 점수: 1.00
⚡ 캐시 HIT (유사도 1.00) [source=pdf_pre_cache]
(캐시 응답)
흙은 「식물방역법」제10조에 따라 수입금지품목이므로 국내 반입이 불가합니다.
- 흙이란 암석 등이 풍화되어 분해된 것으로 유기질이 혼입된 지구표면의
혼합물로 동식물에 큰 피해를 줄 수 있는 각종 병해충이 서식할 가능성이
높습니다.
그러나 시험연구용이나 정부가 인정하는 국제박람회용으로 제공하기 위한
경우 등 특정 상황에서 일정한 요건을 갖추어 농림축산식품부장관의
허가를 받아야만 수입이 가능합니다.
- 흙은 HS부호 2530.90-9099호에 분류되며, 관세는 3%, 부가가치세 과세
대상이며 세관장 확인대상(식물방역법)입니다.
흙에 해당하지 않는 물품
‣ 도토, 인광, 규조토, 보크사이트 등 공업용·화장품 또는 의료용으로 사용되는 것
‣ 유기물이 혼입되어 있지 않는 모래, 자갈
‣ 바다에서 채취된 모래, 자갈
‣ 피트모스, 코코피트(코코넛 허스크), 이끼 등 → 흙으로는 보지 아니하나
식물방역법 대상 물품임
※ 피트모스(Peat Moss) : 늪지 식물이 습지의 바닥에 퇴적되어 부식한 것 \
관련법령
● 식물방역법 제10조제1항제3호, 제2항제1호
PART 1 통 관|219

[표 데이터]

통
관


In [148]:
print(ask("자진신고를 하지 않으면 어떻게 되나요?"))


👤 사용자 질문: 자진신고를 하지 않으면 어떻게 되나요?
📊 유사도 점수: 1.00
⚡ 캐시 HIT (유사도 1.00) [source=pdf_pre_cache]
(캐시 응답)
자진신고 불이행시에는 사후납부 및 면세점 반품 배제, 40%의 가산세 부과 등 불이익이 따
르며 밀수입의 고의가 있다고 판단되는 경우 처벌받을 수 있습니다. ※ 1人기본면세범위
($800)는 자진신고 여부와 관계없이 적용됨
◆ 세금 사후납부 불허
수입물품은 세금납부 또는 납세담보 제공 후 보세구역에서 반출 가능한 것이 원칙이나, 여행자휴대
품의 경우 휴대품을 신고한 국내거주 내국인 여행자에 한하여 예외적으로 사후납부가 허용(단, 체납자,
우범여행자, 만19세 미만자는 제외)되므로, 신고불이행시에는 사후납부가 불허됨
⇨ 여행자및승무원휴대품통관에관한고시 제58조
◆ 면세점 반품 불가
해외에서 취득한 물품(국내면세점 구매후 재반입물품 포함) 중 면세범위 초과분은 세관에 자진신고하여야
하며, 면세점 구매물품을 구매자가 직접 휴대입국하여 교환, 환불 요청시에는 세관에 휴대품 신고 및
유치해야 하므로, 신고불이행시 면세점 반품은 불가함
⇨ 여행자및승무원휴대품통관에관한고시 제6조 / 보세판매장운영에관한고시 제19조
◆ 미신고 가산세 부과
여행자 및 승무원이 신고대상물품을 신고하지 아니한 경우 해당 물품에 대하여 납부할 세액(관세+내국세)의
100분의 40(최대 60*)에 해당하는 금액을 가산세로 징수함
* 입국일 기준으로 소급하여 최근 2년 이내에 2회 이상 미신고 가산세를 징수한 경우
⇨ 관세법 제241조 제5항 제1호, 여행자및승무원휴대품통관에관한고시 제6조
◆ 밀수입죄로 처벌
단순 신고불이행의 범주에서 벗어나 대리반입, 이중바닥이나 신변에 은닉 등 밀수입의 고의가 현저하
다고 인정되는 경우에는 밀수입죄의 미수범으로 처벌 가능(통고처분, 조사의뢰)
⇨ 관세법 제269조 내지 제270조, 제274조, 제311조 등
◆ FTA 협정세율 적용 배제
협정세율은 수입자의 신청에 의해 심사 후 적용가능한 

In [150]:
print(ask("미국의 쇼핑 사이트에서 한화 약 300만원 정도 되는 고급 핸드백을 구입하려고 하는데 납부하여야 하는 세액이 얼마나 되는지요?"))


👤 사용자 질문: 미국의 쇼핑 사이트에서 한화 약 300만원 정도 되는 고급 핸드백을 구입하려고 하는데 납부하여야 하는 세액이 얼마나 되는지요?
📊 유사도 점수: 0.62
⚡ 캐시 HIT (유사도 0.62) [source=pdf_pre_cache]
(캐시 응답)
고급 핸드백은 개별소비세 부과대상물품으로 관세, 개별소비세, 교육세,
부가가치세가 부과됩니다. 고급 핸드백의 경우 물품가격 중 기준가격
(200만원)을 초과하는 부분의 가격에 100분의 20을 곱한 금액이 개별
소비세로 부과됩니다.
각각의 세액을 계산해보면, 물품가격+운임+보험료를 과세가격이라고 한다면,
관세=과세가격×8%, 개별소비세=(관세 과세가격+관세-200만원)×20%,
교육세=개별소비세×30%, 부가가치세=(과세가격+관세+개별소비세+교육세)×
10%가 부과됩니다.
관련법령
● 관세법 제30조(과세가격결정의 원칙)
PART 1 통 관|231

[표 데이터]

통
관
